# Scikit-Learn's new integration with Pandas

In [1]:
import pandas as pd
import numpy as np

In [4]:
train = pd.read_csv('https://raw.githubusercontent.com/DunderData/Tutorials/'
                    'master/Machine%20Learning%20Tutorials/'
                    'From%20Pandas%20to%20Scikit-Learn%E2%80%8A-%E2%80%8AA%20new%20exciting%C2%A0workflow/'
                    'data/housing/train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
train.shape

(1460, 81)

### Удаление зависимой переменной из обучающего набора
Зависимой переменной является `SalesPrice`, которую мы превратим в массив меток. Он будет использован позднее при обучении модели.

In [7]:
y = train.pop('SalePrice').values

## Кодировка отдельного столбца со строковыми значениями
Для начала закодируем отдельный столбец со строковыми значениями `HouseStyle`, который содержит значения, описывающие внешний облик дома. Давайте выведем частоту каждого строкового значения.

In [9]:
vc = train['HouseStyle'].value_counts()
vc

1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: HouseStyle, dtype: int64

In [10]:
len(vc)

8

## Scikit-Learn – только двумерные данные
Большинство моделей scikit-Learn требуют, чтобы данными были строго двумерными. Если мы выбираем столбец `train['HouseStyle']`, технически будет создан объект Series, который является одномерным объектом данных. Можно заставить библиотеку Pandas создать объект DataFrame с единственным столбцом, передав список, состоящий из одного элемента, в квадратные скобки:

In [12]:
hs_train = train[['HouseStyle']].copy()
hs_train.ndim

2

## Импортируем класс, создаем экземпляр класса — модель, обучаем модель – трехэтапный процесс работы с моделью
Интерфейс библиотеки scikit-learn совместим со всеми моделями машинного обучения и использует трехэтапный процесс обучения на данных

1. Импортируем нужный нам класс из соответствующего модуля.
1. Создаем экземпляр класса – модель, возможно изменив значения гиперпараметров по умолчанию.
1. Обучаем модель на данных. Возможно при необходимости мы преобразуем данные в новое пространство.

Ниже мы импортируем класс `OneHotEncoder`, создаем экземпляр класса, убеждаемся в том, что мы получаем плотный (а не разреженный) массив, наконец, выполняем дамми-кодирование нашего отдельного столбца с помощью метода `.fit_transform()`.

In [18]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
hs_train_transformed = ohe.fit_transform(hs_train)
hs_train_transformed

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

Как и ожидалось, каждое уникальное значение становится отдельным бинарным признаком.

In [20]:
hs_train_transformed.shape

(1460, 8)

## У нас NumPy массив. Где имена столбцов?
Отметим, что полученный нами объект представляет собой массив NumPy, а не датафрейм pandas. Библиотека scikit-learn изначально не предполагала непосредственной интеграции с библиотекой pandas. Все объекты библиотеки pandas конвертируются в массивы NumPy под капотом и именно они всегда возвращаются после выполнения преобразования.

Однако мы все же можно извлечь имя столбца из объекта `OneHotEncoder` c помощью метода `.get_feature_names()`.

In [22]:
feature_names = ohe.get_feature_names()
feature_names

array(['x0_1.5Fin', 'x0_1.5Unf', 'x0_1Story', 'x0_2.5Fin', 'x0_2.5Unf',
       'x0_2Story', 'x0_SFoyer', 'x0_SLvl'], dtype=object)

## Проверка корректности первой строки данных
Полезной привычкой будет проверка корректной работы модели. Давайте выведем первую строку преобразованных данных.

In [25]:
row0 = hs_train_transformed[0]
row0

array([0., 0., 0., 0., 0., 1., 0., 0.])

Шестое значение массива было закодировано единицей. Используя логическое индексирование, можно выяснить имя переменной.

In [26]:
feature_names[row0 == 1]

array(['x0_2Story'], dtype=object)

Теперь убедимся, что первое значение в нашем исходном столбце является тем же самым.

In [28]:
hs_train.values[0]

array(['2Story'], dtype=object)

### Используем метод `inverse_transform()` для автоматизации данной операции
Как и у большинства классов-трансформеров, у объекта `OneHotEncoder` есть метод `inverse_transform()`, который вернет вам исходные данные. Здесь мы должны обернуть `row0` в список, чтобы получить двумерный массив.

In [30]:
ohe.inverse_transform([row0])

array([['2Story']], dtype=object)

Мы можем проверить все значения сразу, выполнив инвертирование всего преобразованного массива.

In [32]:
hs_inv = ohe.inverse_transform(hs_train_transformed)
hs_inv

array([['2Story'],
       ['1Story'],
       ['2Story'],
       ...,
       ['2Story'],
       ['1Story'],
       ['1Story']], dtype=object)

In [33]:
np.array_equal(hs_inv, hs_train.values)

True

## Применение преобразования к тестовому набору
Любое преобразование, применяемое к обучающему набору, должно быть применено и к тестовому набору. Давайте прочитаем тестовый набор данных, извлечем тот же самый столбец и применим к нему наше преобразование.

In [35]:
test = pd.read_csv('https://raw.githubusercontent.com/DunderData/Tutorials/'
                    'master/Machine%20Learning%20Tutorials/'
                    'From%20Pandas%20to%20Scikit-Learn%E2%80%8A-%E2%80%8AA%20new%20exciting%C2%A0workflow/'
                    'data/housing/test.csv')
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [36]:
hs_test = test[['HouseStyle']].copy()
hs_test_transformed = ohe.transform(hs_test)
hs_test_transformed

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

По итогам дамми-кодирования должно получиться 8 столбцов, что мы и видим.

In [38]:
hs_test_transformed.shape

(1459, 8)

Этот пример работает исправно, однако существует множество случаев, когда мы можем столкнуться с проблемами. Они будут рассмотрены ниже.

## Проблема No1 – Новые категории в тестовом наборе
Что случится, если в тестовом наборе попадется такая категория переменной `HouseStyle`, уникальна только для него? Например, значение `3Story`. Давайте заменим значение переменной `HouseStyle` в первой строке тестового набора и попытаемся выполнить дамми-кодирование.

In [41]:
hs_test = test[['HouseStyle']].copy()
hs_test.iloc[0, 0] = '3Story'
hs_test.head(3)

,HouseStyle
0,3Story
1,1Story
2,2Story


In [42]:
ohe.transform(hs_test)

ValueError: Found unknown categories ['3Story'] in column 0 during transform

## Error: Unknown Category

По умолчанию энкодер выдаст ошибку. Вероятно, это то, что нужно, ведь нам необходимо знать, есть ли в тестовом наборе новые категории. Если такая проблема возникла, то необходимо более детальное исследование. Сейчас же мы проигнорируем эту проблему и соответствующее наблюдение закодируем строкой, состоящей из нулей, задав для параметра `handle_unknown значение` 'ignore' при создании экземпляра класса `OneHotEncoder`.

In [44]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe.fit(hs_train)

hs_test_transformed = ohe.transform(hs_test)
hs_test_transformed

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

Теперь проверим, состоит ли первая строка из одних нулей.

In [46]:
hs_test_transformed[0]

array([0., 0., 0., 0., 0., 0., 0., 0.])

## Проблема No2 – Пропущенные значения в тестовом наборе

Если тестовый набор содержит пропущенные значения (NaN или None), то они будут проигнорированы при условии, что для параметра `handle_unknown` задано значение 'ignore'. Давайте заменим первые два значения переменной `HouseStyle` в тестовом наборе на пропуски.

In [50]:
hs_test = test[['HouseStyle']].copy()
hs_test.iloc[0, 0] = np.nan
hs_test.iloc[1, 0] = None
hs_test.head(4)

,HouseStyle
0,NaN
1,None
2,2Story
3,2Story


## Проблема No3 – Пропущенные значения в обучающе наборе
Пропущенные значения в обучающей выборке являются большей проблемой. На данный момент мы не можем обучить `OneHotEncoder` на данных, содержащих пропуски.

In [52]:
hs_train = hs_train.copy()
hs_train.iloc[0, 0] = np.nan
hs_train.head(3)

,HouseStyle
0,NaN
1,1Story
2,2Story


In [53]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe.fit_transform(hs_train)

ValueError: Input contains NaN

В данной ситуации было бы воспользоваться той же самой возможностью проигнорировать пропуски, как в примере с тестовым набором выше. Но на данный момент такой возможности нет, поэтому необходимо их импутировать.

## Необходимость импутации пропущенных значений

Теперь необходимо заполнить пропущенные значения. Класс `Imputer` из модуля preprocessing объявлен устаревшим. Новый модуль `impute` пришел ему на замену со своим классом `SimpleImputer` и новой стратегией 'constant'. По умолчанию при использовании этой стратегии пропуски будут заменены на строку 'missing_value'. Но есть возможность заполнять пропуски любыми значениями с помощью параметра fill_value.

In [56]:
hs_train = train[['HouseStyle']].copy()
hs_train.iloc[0, 0] = np.nan

from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='constant', fill_value='MISSING')
hs_train_imputed = si.fit_transform(hs_train)
hs_train_imputed

array([['MISSING'],
       ['1Story'],
       ['2Story'],
       ...,
       ['2Story'],
       ['1Story'],
       ['1Story']], dtype=object)

Вот теперь мы можем выполнить дамми-кодирование, как уже делали это ранее.

In [58]:
hs_train_transformed = ohe.fit_transform(hs_train_imputed)
hs_train_transformed

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

Стоит отметить, что теперь в наборе данных присутствует дополнительный столбец и дополнительное имя столбца.

In [60]:
hs_train_transformed.shape

(1460, 9)

In [ ]:
ohe.get_feature_names()

array(['x0_1.5Fin', 'x0_1.5Unf', 'x0_1Story', 'x0_2.5Fin', 'x0_2.5Unf',
       'x0_2Story', 'x0_MISSING', 'x0_SFoyer', 'x0_SLvl'], dtype=object)

## Больше о методе `fit_transform()`

Для всех моделей метод `fit_transform()` сначала вызывает метод `fit()`, а затем вызывает метод `transform()`. Метод `fit()` вычисляет то, что будет использовано в ходе преобразования. Например, если бы мы задали для `SimpleImputer` стратегию 'mean', то при вызове метода `fit()` мы бы вычисляли и сохраняли среднее значение по каждому признаку. А метод `transform()` использует эти средние значения для заполнения пропусков в каждом соответствующем столбце и возвращает преобразованный массив.

`OneHotEncoder` работает аналогично. Во время выполнения метода `fit()`, он находит все уникальные значения для каждого столбца и сохраняет их. Когда вызывается метод `transform()`, сохраненные уникальные значения для каждого столбца используются для создания бинарных признаков.

### Применение нескольких преобразований к тестовому набору
Кроме того, мы можем вручную применить каждое из описанных выше преобразований в следующем порядке:

In [63]:
hs_test = test[['HouseStyle']].copy()
hs_test.iloc[0, 0] = 'unique value to test set'
hs_test.iloc[1, 0] = np.nan

hs_test_imputed = si.transform(hs_test)
hs_test_transformed = ohe.transform(hs_test_imputed)
hs_test_transformed.shape

(1459, 9)

## Применение конвейера

Библиотека scikit-learn предлагает класс `Pipeline`, который принимает список преобразований и выполняет их последовательно. Кроме того, в него можно поместить модель машинного обучения в качестве конечного класса. Ниже приведен пример простой импутации и дамми- кодирования.

In [65]:
from sklearn.pipeline import Pipeline

Каждый этап конвейера представляет собой кортеж, состоящий из двух элементов – названия шага и экземпляра класса. Результат предыдущего этапа будет входными данными для последующего этапа.

In [67]:
si_step = ('si', SimpleImputer(strategy='constant', fill_value='MISSING'))
ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
steps = [si_step, ohe_step]

pipe = Pipeline(steps)

hs_train = train[['HouseStyle']].copy()
hs_train.iloc[0, 0] = np.nan

hs_transformed = pipe.fit_transform(hs_train)
hs_transformed.shape

(1460, 9)

Преобразования для тестового набора легко выполняются с помощью соответствующего этапа конвейера, для этого нужно просто передать тестовый набор в метод `transform()`.

In [69]:
hs_test = test[['HouseStyle']].copy()
hs_test_transformed = pipe.transform(hs_test)
hs_test_transformed.shape

(1459, 9)

## Почему для тестового набора мы вызываем только метод `transform()`?

Для выполнения преобразований тестового набора нам нужно вызвать метод `transform()`, а не `fit_transform()`, поскольку библиотека scikit-learn уже нашла всю необходимую информацию, которая ей понадобиться для преобразования любого другого набора, содержащего те же самые имена столбцов.

In [71]:
ohe.get_feature_names()

array(['x0_1.5Fin', 'x0_1.5Unf', 'x0_1Story', 'x0_2.5Fin', 'x0_2.5Unf',
       'x0_2Story', 'x0_MISSING', 'x0_SFoyer', 'x0_SLvl'], dtype=object)

## Выполнение преобразований для нескольких столбцов со строковыми значениями

Выполнить преобразование нескольких столбцов со строковыми значениями так же просто. Необходимо выбрать нужные столбцы и передать их в тот же самый конвейер.

In [73]:
string_cols = ['RoofMatl', 'HouseStyle']
string_train = train[string_cols]
string_train.head(3)

,RoofMatl,HouseStyle
0,CompShg,2Story
1,CompShg,1Story
2,CompShg,2Story


In [74]:
string_train_transformed = pipe.fit_transform(string_train)
string_train_transformed.shape

(1460, 16)

### Обращение к отдельным этапам конвейера

При необходимости можно обратиться к отдельному этапу конвейера по его имени с помощью атрибута `named_steps`. В этом примере обратимся к `OneHotEncoder`, чтобы вывести названия признаков.

In [76]:
ohe = pipe.named_steps['ohe']
ohe.get_feature_names()

array(['x0_ClyTile', 'x0_CompShg', 'x0_Membran', 'x0_Metal', 'x0_Roll',
       'x0_Tar&Grv', 'x0_WdShake', 'x0_WdShngl', 'x1_1.5Fin', 'x1_1.5Unf',
       'x1_1Story', 'x1_2.5Fin', 'x1_2.5Unf', 'x1_2Story', 'x1_SFoyer',
       'x1_SLvl'], dtype=object)

## Использование нового `ColumnTransformer` для отбора столбцов

Класс `ColumnTransformer` (входящий в новый модуль compose) позволяет определить, для каких столбцов нужно выполнить определенные преобразования. Категориальные признаки мы почти всегда преобразовываем иначе, чем количественные.

В настоящее время класс `ColumnTransformer` проходит испытание, а значит его функционал может измениться в будущем. ColumnTransformer принимает список трехэлементных кортежей. Первое значение в кортеже является именем самого кортежа, второе – экземпляр класса, третье – список столбцов, которые нужно преобразовать. Кортеж будет выглядеть следующим образом:

```('name', SomeTransformer(parameters), columns)```

Столбцы необязательно должны быть именами столбцов, вы можете использовать целочисленные индексы столбцов, массив булевых значений и даже функцию, которая принимает в качестве аргумента весь DataFrame и возвращает набор столбцов.

Также предусмотрена возможность использовать массив NumPy вместе с `ColumnTransformer`.

## Передаем конвейер в `ColumnTransformer`

Мы даже можем передать конвейер с множетвом преобразований в `ColumnTransformer`, что мы и сделаем ниже, поскольку нам нужно применить несколько преобразований к столбцам со строковыми значениями.

Ниже мы выполним ту же самую импутацию и дамми-кодирование, что и в предыдущих примерах, но теперь с использованием ColumnTransformer. При этом конвейер остается тем же самым, что и прежде, просто к имени каждого этапа добавим префикс `cat`. В следующем разделе мы добавим дополнительный конвейер для количественных столбцов.

In [80]:
from sklearn.compose import ColumnTransformer

cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value='MISSING'))
cat_ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
cat_steps = [cat_si_step, cat_ohe_step]

cat_pipe = Pipeline(cat_steps)
cat_cols = ['RoofMatl', 'HouseStyle']
cat_transformers = [('cat', cat_pipe, cat_cols)]

ct = ColumnTransformer(transformers=cat_transformers)

## Передаем весь объект DataFrame в `ColumnTransformer`
Экземпляр класса `ColumnTransformer` отбирает нужные нам столбцы, поэтому мы можем просто передать весь датафрейм в метод `fit_transform()`. Нужные столбцы будут выбраны автоматически.

In [82]:
X_cat_transformed = ct.fit_transform(train)
X_cat_transformed

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [83]:
X_cat_transformed.shape

(1460, 16)

Тестовый набор можно преобразовать аналогичным образом.

In [85]:
X_cat_transformed_test = ct.transform(test)
X_cat_transformed_test.shape

(1459, 16)

### Извлечение названий признаков

Придется немного поковыряться, чтобы достать названия признаков. Все классы, выполняющие преобразования, хранятся в атрибуте `named_transformers_`. Ниже мы выбираем наш трансформер. Пока он только один – конвейер под названием 'cat', где 'cat' – это имя, первый элемент в трехэлементном кортеже.

In [88]:
pl = ct.named_transformers_['cat']

Теперь из этого конвейера мы извлекаем объект OneHotEncoder и уже из него можно извлечь имена предикторов.

In [91]:
ohe = pl.named_steps['ohe']
ohe.get_feature_names()

array(['x0_ClyTile', 'x0_CompShg', 'x0_Membran', 'x0_Metal', 'x0_Roll',
       'x0_Tar&Grv', 'x0_WdShake', 'x0_WdShngl', 'x1_1.5Fin', 'x1_1.5Unf',
       'x1_1Story', 'x1_2.5Fin', 'x1_2.5Unf', 'x1_2Story', 'x1_SFoyer',
       'x1_SLvl'], dtype=object)

## Преобразование количественных переменных

Количественным переменным могут потребоваться совершенно другие преобразования. Вместо импутации пропусков константами чаще используется импутация средним значением или медианой. И вместо дамми-кодирования обычно выполняют стандартизацию путем вычитания из исходного значения признака среднего значения и делением полученного результата на стандартное отклонение. Это позволяет улучшить качество регрессионных моделей.

## Работа со всеми количественными признаками

Вместо выбора одного или нескольких столбцов вручную, как мы делали это выше, работая со столбцами, содержащими строковые значения, можно выбрать все количественные переменные. Чтобы это сделать, нужно сперва найти тип каждого признака с помощью аттрибута `dtypes`, после чего проверить, имеет ли данный признак тип object. Атрибут `dtypes` позволяет выяснить тип каждого столбца. Мы можем использовать его для поиска столбцов с числовыми или строковыми значениями. Библиотека pandas хранит все столбцы со строковыми значениями как столбцы типа object.

In [94]:
train.dtypes.head()

Id               int64
MSSubClass       int64
MSZoning        object
LotFrontage    float64
LotArea          int64
dtype: object

С помощью атрибута `kind` выводим тип каждой переменной в виде одной буквы.

In [96]:
from operator import attrgetter

In [104]:
train.dtypes.apply(attrgetter('kind'))

Id               i
MSSubClass       i
MSZoning         O
LotFrontage      f
LotArea          i
                ..
MiscVal          i
MoSold           i
YrSold           i
SaleType         O
SaleCondition    O
Length: 80, dtype: object

In [106]:
# kinds = np.array([dt.kind for dt in train.dtypes])
kinds = train.dtypes.apply(attrgetter('kind')).values

In [107]:
kinds[:5]

array(['i', 'i', 'O', 'f', 'i'], dtype=object)

Предположим, что все количественные предикторы не относятся к типу object. Исходя из этого, мы можем извлечь категориальные признаки.

In [109]:
all_columns = train.columns.values
is_num = kinds != 'O'
num_cols = all_columns[is_num]
num_cols[:5]

array(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual'],
      dtype=object)

In [110]:
cat_cols = all_columns[~is_num]
cat_cols[:5]

array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour'],
      dtype=object)

Как только количественные признаки определены, мы вновь можем воспользоваться классом `ColumnTransformer`.

In [112]:
from sklearn.preprocessing import StandardScaler

num_si_step = ('si', SimpleImputer(strategy='median'))
num_ss_step = ('ss', StandardScaler())
num_steps = [num_si_step, num_ss_step]

num_pipe = Pipeline(num_steps)
num_transformers = [('num', num_pipe, num_cols)]

ct = ColumnTransformer(transformers=num_transformers)

X_num_transformed = ct.fit_transform(train)
X_num_transformed.shape

(1460, 37)

## Передаем конвейер с преобразованиями для категориальных признаков и конвейер с преобразования для количественных признаков в `ColumnTransformer`

С помощью `ColumnTransformer` для каждого типа столбцов можно применить отдельные преобразования. Нам нужно определиться с операциями преобразования для категориальных признаков и операциями преобразования для количественных признаков. После этого создаем отдельный конвейер для категориальных признаков, отдельный конвейер для количественных признаков, затем используем `ColumnTransformer` для независимого преобразования признаков. Преобразования в обоих конвейерах будут выполняться **параллельно**, а их результаты будут сконкатенированы.

In [114]:
transformers = [('cat', cat_pipe, cat_cols),
                ('num', num_pipe, num_cols)]

ct = ColumnTransformer(transformers=transformers)

X = ct.fit_transform(train)
X.shape

(1460, 305)

# Machine Learning

Целью всех вышеописанных этапов была подготовка данных для последующего машинного обучения. Мы можем создать итоговый конвейер и добавить в него в качестве последнего этапа модель машинного обучения. Первым этапом этого конвейера будет преобразование данных, описанное выше. В качестве зависимой переменной $y$, как было принято выше, берется переменная `SalePrice`. Здесь мы просто используем метод `fit` вместо `fit_transform`, поскольку итоговым этапом конвейера будет модель машинного обучения, которая не производит никаких преобразований.

In [116]:
from sklearn.linear_model import Ridge

ml_pipe = Pipeline([('transform', ct), ('ridge', Ridge())])
ml_pipe.fit(train, y)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('si',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'Bld...
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBa

In [117]:
ml_pipe.score(train, y)

0.9220545988101001

Оценить качество модели можно, например, с помощью метода `score()`, возвращающего значение R-квадрат.

## Перекрестная проверка

Разумеется, оценка модели на обучающем наборе данных бесполезна. Чтобы лучше понять, как модель поведет себя на неизвестных ей данных, нужно провести k-блочную перекрестную проверку. Для воспроизводимости результатов нужно задать значение random_state.

In [120]:
from sklearn.model_selection import KFold, cross_val_score
kf = KFold(n_splits=5, shuffle=True, random_state=123)

cross_val_score(ml_pipe, train, y, cv=kf).mean()

0.8133920019569674

## Отбор наилучших значений гиперпараметров с помощью поиска по решетке
Решетчатый поиск в библиотеке scikit-learn требует передачи словаря, ключами которого будут названия гиперпараметров, а значениями – списки значений этих гиперпараметров. При работе с конвейером мы должны взять имя этапа с двумя символами нижнего подчеркивания на конце и после этого указать имя гиперпараметра. Если мы работаем с многоуровневым конвейером, как в примере ниже, то тогда двойное нижнее подчеркивание должно разделять имена всех уровней (`transform__num__si__strategy`, здесь мы работаем с уровнями `transform`, `num` и `si`), пока не будет достигнуто название гиперпараметра экземпляра класса (в данном случае название гиперпараметра `strategy` объект `si`, экземпляр класса `SimpleImputer`), по значениям которого должен быть осуществлен поиск (в данном случае, как можно увидеть ниже, поиск должен быть осуществлен по значениям 'mean' и 'median').

In [122]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'transform__num__si__strategy': ['mean', 'median'],
    'ridge__alpha': [.001, 0.1, 1.0, 5, 10, 50, 100, 1000],
}
gs = GridSearchCV(ml_pipe, param_grid, cv=kf)

In [123]:
gs.fit(train, y)
gs.best_params_

{'ridge__alpha': 10, 'transform__num__si__strategy': 'median'}

In [124]:
gs.best_score_

0.8190367464419683

## Представление результатов решетчатого поиска в виде датафрейма pandas
Результаты решетчатого поиска хранятся в атрибуте `cv_results_`. Он представляет собой словарь, а значит его можно представить в виде объекта DataFrame для удобства чтения.

In [126]:
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200

In [127]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ridge__alpha,param_transform__num__si__strategy,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.094233,0.032500,0.022859,0.003429,0.001,mean,"{'ridge__alpha': 0.001, 'transform__num__si__strategy': 'mean'}",0.898019,0.024032,0.775950,0.883812,0.662549,0.648872,0.323738,16
1,0.086549,0.033288,0.024504,0.004903,0.001,median,"{'ridge__alpha': 0.001, 'transform__num__si__strategy': 'median'}",0.898015,0.024086,0.775946,0.883812,0.662624,0.648897,0.323717,15
2,0.069132,0.005389,0.020773,0.001036,0.1,mean,"{'ridge__alpha': 0.1, 'transform__num__si__strategy': 'mean'}",0.897111,0.779393,0.800366,0.884558,0.653398,0.802965,0.087697,12
3,0.080703,0.012485,0.027079,0.009833,0.1,median,"{'ridge__alpha': 0.1, 'transform__num__si__strategy': 'median'}",0.897114,0.779408,0.800366,0.884561,0.653472,0.802984,0.087672,11
4,0.115897,0.028878,0.065654,0.047379,1,mean,"{'ridge__alpha': 1.0, 'transform__num__si__strategy': 'mean'}",0.891715,0.809381,0.822107,0.876908,0.666710,0.813364,0.079720,8
5,0.114244,0.025624,0.031586,0.004470,1,median,"{'ridge__alpha': 1.0, 'transform__num__si__strategy': 'median'}",0.891728,0.809413,0.822128,0.876924,0.666768,0.813392,0.079704,7
6,0.131342,0.025240,0.049480,0.026445,5,mean,"{'ridge__alpha': 5, 'transform__num__si__strategy': 'mean'}",0.889795,0.820456,0.820334,0.871716,0.684609,0.817382,0.071892,4
7,0.097935,0.018528,0.023620,0.004404,5,median,"{'ridge__alpha': 5, 'transform__num__si__strategy': 'median'}",0.889804,0.820471,0.820383,0.871755,0.684598,0.817402,0.071905,3
8,0.075922,0.017133,0.025066,0.008326,10,mean,"{'ridge__alpha': 10, 'transform__num__si__strategy': 'mean'}",0.890470,0.824359,0.819779,0.871443,0.689068,0.819024,0.070385,2
9,0.070075,0.006050,0.019205,0.001232,10,median,"{'ridge__alpha': 10, 'transform__num__si__strategy': 'median'}",0.890488,0.824358,0.819844,0.871495,0.688998,0.819037,0.070422,1


## Создание пользовательского трансформера, выполняющего основные преобразования
У вышеописанного подхода к обработке данных есть недостатки. Например, было бы удобно, если бы `OneHotEncoder` позволял игнорировать пропущенные данные в ходе применения метода `fit()`. Ведь можно закодировать пропущенные значения, просто заполнив строку нулями. Но сейчас метод требует, чтобы пропуски были заменены некоторым строковым значением, которое потом будет представлено в виде отдельного бинарного признака.

### Редкие категории
Строковые значения признака, которые встретились в обучающем наборе несколько раз, вряд ли будут полезными при валидации модели. Иногда их желательно закодировать так, как если бы эти значения были пропусками.

### Написание собственного пользовательского класса
В документации scikit-learn дается справка по написанию собственного класса. Класс `BaseEstimator`, расположенный внутри модуля base предлагает методы `get_params` и `set_params`. Метод `set_params` необходим при выполнении поиска по решетке. Можно написать собственный класс или наследовать из `BaseEstimator`. Существует также класс `TransformerMixin`, однако он позволяет лишь написать метод `fit_transform`.

Класс `BasicTransformer`, который мы сейчас создадим, выполнет следующие операции:

* заполняет пропуски в количественных признаках средним значением или медианой;
* стандартизирует все количественные признаки;
* выполняет дамми-кодирование для признаков со строковыми значениями;
* не заполняет пропуски в категориальных признаках, а при выполнении дамми-кодирования заменяет пропущенное значение строкой, состоящей из нулей;
* игнорирует новые строковые значения (категории) в тестовом наборе;
* позволяет устанавливать порог частоты строковых значений, ниже которого строковое значение по итогам дамми-кодирования будет представлено строкой из нулей.
* работает только с датафреймами pandas, находится в разработке и поэтому может некорректно работать c некоторыми наборами данных
* назван basic, потому что содержит все основные преобразования, которые, как правило, применяются для большинства наборов данных.

In [140]:
from sklearn.base import BaseEstimator

class BasicTransformer(BaseEstimator):
    
    def __init__(self, cat_threshold=None, num_strategy='median', return_df=False):
        # store parameters as public attributes
        self.cat_threshold = cat_threshold
        
        if num_strategy not in ['mean', 'median']:
            raise ValueError('num_strategy must be either "mean" or "median"')
        self.num_strategy = num_strategy
        self.return_df = return_df
        
    def fit(self, X, y=None):
        # Assumes X is a DataFrame
        self._columns = X.columns.values
        
        # Split data into categorical and numeric
        self._dtypes = X.dtypes.values
        self._kinds = np.array([dt.kind for dt in X.dtypes])
        self._column_dtypes = {}
        is_cat = self._kinds == 'O'
        self._column_dtypes['cat'] = self._columns[is_cat]
        self._column_dtypes['num'] = self._columns[~is_cat]
        self._feature_names = self._column_dtypes['num']
        
        # Create a dictionary mapping categorical column to unique values above threshold
        self._cat_cols = {}
        for col in self._column_dtypes['cat']:
            vc = X[col].value_counts()
            if self.cat_threshold is not None:
                vc = vc[vc > self.cat_threshold]
            vals = vc.index.values
            self._cat_cols[col] = vals
            self._feature_names = np.append(self._feature_names, col + '_' + vals)
            
        # get total number of new categorical columns    
        self._total_cat_cols = sum([len(v) for col, v in self._cat_cols.items()])
        
        # get mean or median
        self._num_fill = X[self._column_dtypes['num']].agg(self.num_strategy)
        return self
        
    def transform(self, X):
        # check that we have a DataFrame with same column names as the one we fit
        if set(self._columns) != set(X.columns):
            raise ValueError('Passed DataFrame has different columns than fit DataFrame')
        elif len(self._columns) != len(X.columns):
            raise ValueError('Passed DataFrame has different number of columns than fit DataFrame')
            
        # fill missing values    
        X_num = X[self._column_dtypes['num']].fillna(self._num_fill)
        
        # Standardize numerics
        std = X_num.std()
        X_num = (X_num - X_num.mean()) / std
        zero_std = np.where(std == 0)[0]
        
        # If there is 0 standard deviation, then all values are the same. Set them to 0.
        if len(zero_std) > 0:
            X_num.iloc[:, zero_std] = 0
        X_num = X_num.values
        
        # create separate array for new encoded categoricals
        X_cat = np.empty((len(X), self._total_cat_cols), dtype='int')
        i = 0
        for col in self._column_dtypes['cat']:
            vals = self._cat_cols[col]
            for val in vals:
                X_cat[:, i] = X[col] == val
                i += 1
                
        # concatenate transformed numeric and categorical arrays
        data = np.column_stack((X_num, X_cat))
        
        # return either a DataFrame or an array
        if self.return_df:
            return pd.DataFrame(data=data, columns=self._feature_names)
        else:
            return data
    
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)
    
    def get_feature_names(self):
        return self._feature_names

## Применение собственного класса `BasicTransformer`
Пользовательский класс `BasicTransformer` должен быть пригодным для использования точно так же, как и любой другой класс библиотеки scikit-learn. Мы можем создать экземпляр класса `BasicTransformer` и с его помощью преобразовать данные.

In [142]:
bt = BasicTransformer(cat_threshold=3, return_df=True)
train_transformed = bt.fit_transform(train)
train_transformed.head(3)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_RL,MSZoning_RM,MSZoning_FV,MSZoning_RH,MSZoning_C (all),Street_Pave,Street_Grvl,Alley_Grvl,Alley_Pave,LotShape_Reg,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,KitchenQual_Ex,KitchenQual_Fa,Functional_Typ,Functional_Min2,Functional_Min1,Functional_Mod,Functional_Maj1,Functional_Maj2,FireplaceQu_Gd,FireplaceQu_TA,FireplaceQu_Fa,FireplaceQu_Ex,FireplaceQu_Po,GarageType_Attchd,GarageType_Detchd,GarageType_BuiltIn,GarageType_Basment,GarageType_CarPort,GarageType_2Types,GarageFinish_Unf,GarageFinish_RFn,GarageFinish_Fin,GarageQual_TA,GarageQual_Fa,GarageQual_Gd,GarageCond_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,PavedDrive_Y,PavedDrive_N,PavedDrive_P,Fence_MnPrv,Fence_GdPrv,Fence_GdWo,Fence_MnWw,MiscFeature_Shed,SaleType_WD,SaleType_New,SaleType_COD,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_CWD,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_Abnorml,SaleCondition_Family,SaleCondition_Alloca,SaleCondition_AdjLand
0,-1.730272,0.073350,-0.220799,-0.207071,0.651256,-0.517023,1.050634,0.878367,0.513928,0.575228,-0.288554,-0.944267,-0.459145,-0.793162,1.161454,-0.120201,0.370207,1.107431,-0.240978,0.78947,1.227165,0.163723,-0.211381,0.911897,-0.950901,1.017250,0.311618,0.35088,-0.751918,0.216429,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,-1.598563,0.138730,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-1.727900,-0.872264,0.460162,-0.091855,-0.071812,2.178881,0.156680,-0.429430,-0.570555,1.171591,-0.288554,-0.641008,0.466305,0.257052,-0.794891,-0.120201,-0.482347,-0.819684,3.947457,0.78947,-0.761360,0.163723,-0.211381,-0.318574,0.600289,-0.107890,0.311618,-0.06071,1.625638,-0.704242,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,-0.488943,-0.614228,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-1.725528,0.073350,-0.084607,0.073455,0.651256,-0.517023,0.984415,0.829930,0.325803,0.092875,-0.288554,-0.301540,-0.313261,-0.627611,1.188943,-0.120201,0.514836,1.107431,-0.240978,0.78947,1.227165,0.163723,-0.211381,-0.318574,0.600289,0.933906,0.311618,0.63151,-0.751918,-0.070337,-0.359202,-0.116299,-0.270116,-0.068668,-0.087658,0.990552,0.138730,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Использование `BasicTransformer` в конвейере
Наш трансформер может быть частью конвейера.

In [144]:
basic_pipe = Pipeline([('bt', bt), ('ridge', Ridge())])
basic_pipe.fit(train, y)
basic_pipe.score(train, y)

0.9035633239411638

Кроме того, мы можем передать наш конвейер в цикл перекрестной проверки, как это уже делали выше.

In [146]:
cross_val_score(basic_pipe, train, y, cv=kf).mean()

0.8157670129480051

Аналогичным образом мы можем разместить конвейер внутри объекта `GridSearchCV`. Оказывается, исключение редких категорий в данном случае не помогает улучшить качество модели. Наилучшее значение немного выросло, возможно в силу несколько иного способа обработки редких категорий.

In [148]:
param_grid = {
    'bt__cat_threshold': [0, 1, 2, 3, 4, 5],
    'ridge__alpha': [.1, 1, 10, 30, 100]
}

gs = GridSearchCV(basic_pipe, param_grid, cv=kf)
gs.fit(train, y)
gs.best_params_

{'bt__cat_threshold': 0, 'ridge__alpha': 10}

In [149]:
gs.best_score_

0.8297473585998103

In [150]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bt__cat_threshold,param_ridge__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.375009,0.149435,0.113036,0.006743,0,0.1,"{'bt__cat_threshold': 0, 'ridge__alpha': 0.1}",0.896662,0.781349,0.803144,0.878337,0.684654,0.808829,0.075833,26
1,0.327850,0.084792,0.122179,0.015732,0,1,"{'bt__cat_threshold': 0, 'ridge__alpha': 1}",0.893991,0.806010,0.832287,0.874838,0.700530,0.821531,0.067956,19
2,1.426267,1.637936,0.204920,0.072928,0,10,"{'bt__cat_threshold': 0, 'ridge__alpha': 10}",0.894996,0.822835,0.835066,0.871813,0.724028,0.829747,0.058787,1
3,0.423252,0.055176,0.201568,0.017539,0,30,"{'bt__cat_threshold': 0, 'ridge__alpha': 30}",0.895582,0.825460,0.831901,0.871506,0.723175,0.829525,0.059091,2
4,0.419776,0.155324,0.210671,0.065491,0,100,"{'bt__cat_threshold': 0, 'ridge__alpha': 100}",0.889236,0.823003,0.822421,0.866925,0.715614,0.823439,0.059746,13
5,0.392862,0.211958,0.205172,0.050289,1,0.1,"{'bt__cat_threshold': 1, 'ridge__alpha': 0.1}",0.884871,0.712477,0.821721,0.863899,0.717456,0.800085,0.072432,30
6,0.372128,0.100454,0.144756,0.034754,1,1,"{'bt__cat_threshold': 1, 'ridge__alpha': 1}",0.888417,0.797304,0.821625,0.866313,0.703418,0.815415,0.064544,22
7,0.380139,0.094461,0.157599,0.027270,1,10,"{'bt__cat_threshold': 1, 'ridge__alpha': 10}",0.894205,0.822060,0.831289,0.870218,0.723553,0.828265,0.058516,10
8,0.412859,0.122898,0.195104,0.063500,1,30,"{'bt__cat_threshold': 1, 'ridge__alpha': 30}",0.895353,0.825277,0.830322,0.870972,0.723009,0.828987,0.059015,5
9,0.306404,0.131580,0.152059,0.053103,1,100,"{'bt__cat_threshold': 1, 'ridge__alpha': 100}",0.889180,0.822979,0.821884,0.866778,0.715578,0.823280,0.059727,15


## Биннинг и преобразование количественных переменных с помощью нового класса `KBinsDiscretizer`
У нас есть несколько признаков, содержащих года. Имеет смысл дискретизировать их и обрабатывать как категориальные переменные. Для решения этой задачи библиотека scikit-learn предложила новый класс `KBinsDiscretizer`. Он не только определяет бины, но и выполняет дамми-кодирование, т.е. каждую полученную категорию (бин) представляет в виде бинарного столбца. Раньше в библиотеке pandas это можно было сделать с помощью функций `cut()` и `qcut()`. Ниже приведен принцип работы класса KBinsDiscretizer на примере признака `YearBuilt`.

In [152]:
from sklearn.preprocessing import KBinsDiscretizer
kbd = KBinsDiscretizer(encode='onehot-dense')
year_built_transformed = kbd.fit_transform(train[['YearBuilt']])
year_built_transformed

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

По умолчанию каждый интервал содержит (приблизительно) равное количество наблюдений. Давайте убедимся в этом.

In [154]:
year_built_transformed.sum(axis=0)

array([292., 274., 307., 266., 321.])

Это мы применили метод 'quantile', выставленный по умолчанию (регулируется с помощью гиперпараметра strategy). Количество создаваемых бинов по умолчанию равно 5 (регулируется с помощью гиперпараметра n_bins). Можно применить метод 'uniform', чтобы разбить интервалы на строго равное количество наблюдений или метод 'kmeans', использующий кластеризацию методом k-средних для определения интервалов.

Теперь посмотрим на границы бинов.

In [157]:
kbd.bin_edges_

array([array([1872. , 1947.8, 1965. , 1984. , 2003. , 2010. ])],
      dtype=object)

## Отдельная обработка всех столбцов с годами с помощью `ColumnTransformer`
Теперь мы возьмем еще один набор признаков, которое требует отдельной обработки и мы выполним его с помощью класса `ColumnTransformer`. Мы добавим еще один этап к нашим предыдущим трансформерам. Кроме того, мы отбросим столбец id, который просто идентифицирует каждое наблюдение.

In [164]:
year_cols = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']
not_year = ~np.isin(num_cols, year_cols + ['Id'])
num_cols2 = num_cols[not_year]

year_si_step = ('si', SimpleImputer(strategy='median'))
year_kbd_step = ('kbd', KBinsDiscretizer(n_bins=3, encode='onehot-dense'))
year_steps = [year_si_step, year_kbd_step]
year_pipe = Pipeline(year_steps)

transformers = [('cat', cat_pipe, cat_cols),
                ('num', num_pipe, num_cols2),
                ('year', year_pipe, year_cols)]

ct = ColumnTransformer(transformers=transformers)
X = ct.fit_transform(train)
X.shape

(1460, 312)

После проведения перекрестной проверки и оценки качества модели очевидно, что все проделанные преобразования не принесли улучшений модели.

In [165]:
ml_pipe = Pipeline([('transform', ct), ('ridge', Ridge())])
cross_val_score(ml_pipe, train, y, cv=kf).mean()

0.8152517065590903

Перебор разного количества бинов может положительно повлиять на качество модели.

In [168]:
import warnings
warnings.filterwarnings('ignore')

param_grid = {
'transform__year__kbd__n_bins': [4, 6, 8, 10], 'ridge__alpha': [.1, .5, 1, 5, 10, 100]
}
# передаем наш конвейер в объект GridSearchCV
gs = GridSearchCV(ml_pipe, param_grid, cv=kf) # выполняем поиск по решетке
gs.fit(train, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=123, shuffle=True),
             estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('si',
                                                                                          SimpleImputer(fill_value='MISSING',
                                                                                                        strategy='constant')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         array(['MSZoning', 'Street', '

In [169]:
# смотрим наилучшие значения гиперпараметров
gs.best_params_

{'ridge__alpha': 10, 'transform__year__kbd__n_bins': 6}

In [170]:
# смотрим наилучшее значение R-квадрат
gs.best_score_

0.8198372622112806

# Применение RobustScaler и FunctionTransformer
Теперь попробуем решить задачу по-другому. Заново загрузим данные, выделим признаки с небольшой отрицательной асимметрией, признаки с небольшой и средней положительной асимметрией и признаки с высокой положительной асимметрией и применим для них соответствующее преобразование, максимизирующее нормальность. Кроме того, для переменных с небольшой и средней положительной асимметрией применим другой тип стандартизации, при котором из каждого исходного значения признака вычитается значение, соответствующее первому квартилю, и полученный результат делится на межквартильный размах (эту стандартизацию можно выполнить с помощью класса `RobustScaler`).

Итак, давайте загрузим данные, создадим массив меток и удалим переменную `Id`.

In [172]:
train = pd.read_csv('https://raw.githubusercontent.com/DunderData/Tutorials/'
                    'master/Machine%20Learning%20Tutorials/'
                    'From%20Pandas%20to%20Scikit-Learn%E2%80%8A-%E2%80%8AA%20new%20exciting%C2%A0workflow/'
                    'data/housing/train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [173]:
y = train.pop('SalePrice').values

In [174]:
# удаляем переменную Id
train.drop('Id', axis=1, inplace=True)

Теперь создадим список количественных признаков и список категориальных признаков.

In [176]:
# выделим категориальные и количественные признаки
cat_columns = train.dtypes[train.dtypes == 'object'].index
num_columns = train.dtypes[train.dtypes != 'object'].index

Вычислим коэффициент асимметрии для количественных признаков.

In [180]:
train[num_columns].skew(0)

MSSubClass        1.407657
LotFrontage       2.163569
LotArea          12.207688
OverallQual       0.216944
OverallCond       0.693067
YearBuilt        -0.613461
YearRemodAdd     -0.503562
MasVnrArea        2.669084
BsmtFinSF1        1.685503
BsmtFinSF2        4.255261
BsmtUnfSF         0.920268
TotalBsmtSF       1.524255
1stFlrSF          1.376757
2ndFlrSF          0.813030
LowQualFinSF      9.011341
GrLivArea         1.366560
BsmtFullBath      0.596067
BsmtHalfBath      4.103403
FullBath          0.036562
HalfBath          0.675897
BedroomAbvGr      0.211790
KitchenAbvGr      4.488397
TotRmsAbvGrd      0.676341
Fireplaces        0.649565
GarageYrBlt      -0.649415
GarageCars       -0.342549
GarageArea        0.179981
WoodDeckSF        1.541376
OpenPorchSF       2.364342
EnclosedPorch     3.089872
3SsnPorch        10.304342
ScreenPorch       4.122214
PoolArea         14.828374
MiscVal          24.476794
MoSold            0.212053
YrSold            0.096269
dtype: float64

Теперь импортируем классы `FunctionTransformer` и `RobustScaler`.

In [182]:
from sklearn.preprocessing import FunctionTransformer, RobustScaler

Теперь выделим списки признаков в зависимости от их коэффициента асимметрии, создадим для каждого списка свой конвейер преобразований и построим модель гребневой регрессии, вновь для проверки качества применив перекрестную проверку.

In [194]:
skewness = train[num_columns].skew(0)

In [232]:
# выделяем список признаков с небольшой отрицательной асимметрией
neg_skew_num_columns = skewness[skewness < 0].index
# выделяем список признаков с высокой положительной асимметрией
high_pos_skew_num_columns = skewness[skewness > np.quantile(skewness, .87)].index
# создадим булев массив
not_neg_high_pos_skew_num_columns = ~np.isin(num_columns, high_pos_skew_num_columns.append(neg_skew_num_columns))
# из списка количественных признаков удалим количественные признаки с небольшой отрицательной и высокой положительной асимметрией
num_columns = num_columns[not_neg_high_pos_skew_num_columns]

In [236]:
# создаем конвейер преобразований для количественных признаков с небольшой отрицательной асимметрией
num_negskew_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('square', FunctionTransformer(np.square, validate=False)),
    ('scaler', StandardScaler())
])

# создаем конвейер преобразований для количественных признаков с небольшой и средней положительной асимметрией
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('log', FunctionTransformer(np.log1p, validate=False)),
    ('scaler', RobustScaler())
])

# создаем конвейер преобразований для количественных признаков с высокой асимметрией
num_highposskew_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('sqrt', FunctionTransformer(np.sqrt, validate=False)),
    ('kbd', KBinsDiscretizer(n_bins=5, encode='onehot-dense'))
])

# создаем конвейер преобразований для категориальных признаков
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='MISSING')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore')) ])

transformers = [
    ('num_negskew', num_negskew_pipe, neg_skew_num_columns), 
    ('num', num_pipe, num_columns),
    ('num_highposskew', num_highposskew_pipe, high_pos_skew_num_columns),
    ('cat', cat_pipe, cat_columns)]

transformer = ColumnTransformer(transformers=transformers)
# добавляем в конвейер новый этап - модель машинного обучения (модель гребневой регрессии)
ml_pipe = Pipeline([
    ('transform', transformer),
    ('ridge', Ridge())]) # выполняем перекрестную проверку, конвейер размещен
# внутри цикла перекрестной проверки
cross_val_score(ml_pipe, train, y, cv=kf).mean()

0.8307940999181334

Теперь попробуем подобрать оптимальные значения гиперпараметров `alpha` и `n_bins` с помощью решетчатого поиска.

In [238]:
# задаем сетку гиперпараметров
param_grid = {
    'ridge__alpha': [.1, .5, 1, 5, 10, 100],
    'transform__num_highposskew__kbd__n_bins': [3, 4, 5, 6, 7] }
# передаем наш конвейер в объект GridSearchCV
gs = GridSearchCV(ml_pipe, param_grid, cv=kf) # выполняем решетчатый поиск
gs.fit(train, y)
# смотрим наилучшие значения гиперпараметров
gs.best_params_

{'ridge__alpha': 10, 'transform__num_highposskew__kbd__n_bins': 7}

In [239]:
# смотрим наилучшее значение R-квадрат
gs.best_score_

0.8474547753257715

## Заключение
Подбор преобразования в зависимости от асимметрии распределения переменной, применение `RobustScaler` и более тщательный перебор значений гиперпараметров `alpha` и `n_bins` позволил улучшить результат.